# Snowflake SQL Alchemy

### Uploading the cleaned PDF data, after using pydantic, pdf metadata and pdf contents into snowflake using Snowflake-SQLAlchemy into our created db

In [ ]:
pip install python-dotenv

In [ ]:
pip install bcrypt

In [ ]:
pip install snowflake-sqlalchemy

In [1]:
#!/usr/bin/env python
import warnings
warnings.filterwarnings("ignore")
from dotenv import load_dotenv
from sqlalchemy import create_engine
import bcrypt
import os
import csv

In [ ]:
def convert_to_url(column_data, bucket_name):
    # Constructing the URL based on the first column data
    trimmed_data = column_data[5:7]
    url = f"https://s3.console.aws.amazon.com/s3/object/{bucket_name}?region=us-east-1&bucketType=general&prefix=Grobid_RR_2024_{trimmed_data}_combined.txt"
    return url

load_dotenv()

u=os.getenv("BUCKET_NAME")

# Open the CSV file for reading and writing
with open('metadata.csv', mode='r') as file:
    reader = csv.DictReader(file)
    fieldnames = reader.fieldnames + ['URL']
    
    # Create a new CSV file with an additional column for the URLs
    with open('metadata_new.csv', mode='w', newline='') as output_file:
        writer = csv.DictWriter(output_file, fieldnames=fieldnames)
        writer.writeheader()
        
        # Iterate over each row in the CSV file
        for row in reader:
            column_data = row['filename']  # Assuming the first column contains the data for conversion
            url = convert_to_url(column_data)
            row['URL'] = url
            
            # Write the modified row to the new CSV file
            writer.writerow(row)

print("Conversion completed. Output saved to output.csv")



### Snowflake Testing environment setup & data upload

In [ ]:
create_test_stage = """CREATE STAGE TEST_URL_STAGING DIRECTORY = ( ENABLE = true );"""

In [2]:
create_test_contents_table = """CREATE OR REPLACE TABLE test_cfa_courses (
        NameOfTopic STRING,
        Title STRING,
        Year INTEGER,
        Level STRING,
        Introduction STRING,
        LearningOutcome STRING,
        LinkToPDF STRING,
        LinkToSummary STRING
        );"""

In [ ]:
create_test_metadata_table = """CREATE OR REPLACE TABLE test_cfa_courses (
        NameOfTopic STRING,
        Title STRING,
        Year INTEGER,
        Level STRING,
        Introduction STRING,
        LearningOutcome STRING,
        LinkToPDF STRING,
        LinkToSummary STRING
        );"""

In [4]:
upload_contents_to_test_stage = """PUT file://..\data\Validation_data.csv @PC_DBT_DB.public.TEST_URL_STAGING;"""
upload_metadata_to_test_stage = """PUT file://..\data\Validation_data.csv @PC_DBT_DB.public.TEST_URL_STAGING;"""

In [5]:
copy_contents_to_test_table = """COPY INTO test_cfa_courses
  FROM @PC_DBT_DB.public.TEST_URL_STAGING
  FILE_FORMAT = (type = csv field_optionally_enclosed_by='"' skip_header = 1)
  PATTERN = 'Validation_data.csv.gz'
  ON_ERROR = 'skip_file';"""

In [ ]:
copy_metadata_to_test_table = """COPY INTO test_cfa_courses
  FROM @PC_DBT_DB.public.TEST_URL_STAGING
  FILE_FORMAT = (type = csv field_optionally_enclosed_by='"' skip_header = 1)
  PATTERN = 'Validation_data.csv.gz'
  ON_ERROR = 'skip_file';"""

### Snowflake Production Environment setup & data upload

In [ ]:
create_prod_stage = """CREATE STAGE URL_STAGING DIRECTORY = ( ENABLE = true );"""

In [ ]:
create_prod_table_query = """CREATE OR REPLACE TABLE cfa_courses (
        NameOfTopic STRING,
        Title STRING,
        Year INTEGER,
        Level STRING,
        Introduction STRING,
        LearningOutcome STRING,
        LinkToPDF STRING,
        LinkToSummary STRING
        );"""

In [ ]:
upload_to_prod_stage = """PUT file://..\data\Validation_data.csv @PC_DBT_DB.public.URL_STAGING;"""

In [ ]:
copy_stage_to_prod_table = """COPY INTO cfa_courses
  FROM @PC_DBT_DB.public.URL_STAGING
  FILE_FORMAT = (type = csv field_optionally_enclosed_by='"' skip_header = 1)
  PATTERN = 'Validation_data.csv.gz'
  ON_ERROR = 'skip_file';"""

In [6]:
load_dotenv()

u=os.getenv("SNOWFLAKE_USER")
p=os.getenv("SNOWFLAKE_PASS")
ai=os.getenv("SNOWFLAKE_ACCOUNTID")


engine = create_engine(
    'snowflake://{user}:{password}@{account_identifier}/'.format(
        user=u,
        password=p,
        account_identifier=ai,
    )
)



try:
    connection = engine.connect()
    connection.execute("USE DATABASE PC_DBT_DB")
    connection.execute("USE WAREHOUSE PC_DBT_WH")
    results = connection.execute(create_table_query)
    results = connection.execute(create_stage)
    results = connection.execute(upload_to_stage)
    results = connection.execute(copy_stage_to_table)

finally:
    print("Done")
    connection.close()
    engine.dispose()

Done
